# Pengantar _Computer Aided Cell Factory Design_
Oleh: **Matin Nuhamunada, M.Sc.** \
_Laboratorium Bioteknologi, Fakultas Biologi, Universitas Gadjah Mada_ \
_matin_nuhamunada@ugm.ac.id_

## Daftar Isi
* [Pengantar](#section1)
* [Setting Up Environment](#section2)
* [Database genome-scale metabolic model](#section3)
* [Eksplorasi dan Simulasi Model](#section4)
* [Manipulasi model & Aplikasi](#section5)
    * [Membuat knock-out](#section5-1)
    * [Mengubah medium](#section5-2)
    * [Mengubah objektif](#section5-3)
    * [Aplikasi: ekspresi heterologous vanilin](#section5-4)
* [What's next? COBRA Plugins & DD Decaf](#section6)
<a name="section1"></a>

## Pengantar
Materi ini ditulis dalam rangka mendampingi workshop Brainstorming BIOS yang diselenggarakan oleh BEM Fakultas Biologi UGM. Workshop ini bertujuan untuk mengenalkan bagaimana CAD (_Computer Aided Design_) diaplikasikan dalam bioteknologi / metabolic engineering. Materi pada workshop ini diringkas dari workshop Metabolic Modeling for Cell Factory Design dari DTU Biosustain, dan selengkapnya dapat diakses di https://biosustain.github.io/cell-factory-design-course/.

[**Slide pengantar workshop dapat diakses di sini.**](cell-factory-design-ugm-2021.pdf)

Karena materi pada workshop ini lebih ke arah praktis, ada baiknya peserta juga mempelajari materi / kuliah [**Transformasi Digital**](https://elok.ugm.ac.id/course/view.php?id=632) (skill dasar Python), [**Bioinformatika**](https://elok.ugm.ac.id/course/view.php?id=1136), dan **Bioteknologi** (Pengantar Systems & Synthetic Biology). Materi kuliah pengantar systems & synthetic biology (archive Kuliah Bioteknologi semester genap 2020) dapat diakses di: http://ugm.id/lectsystembio2020 & http://ugm.id/lectsyntheticbio2020
<a name="section2"></a>

## Setting Up Environment
* Saya merekomendasikan untuk menggunakan OS Ubuntu/Linux (ex: menggunakan [WSL2](https://docs.microsoft.com/en-us/windows/wsl/install-win10))
* Ikuti petunjuk instalasi Conda di: https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html#
* Buat environment sesuai petunjuk di https://biosustain.github.io/cell-factory-design-course/setup/

### Jika ada problem dengan instalasi environment
Jika ada problem dengan instalasi library, coba install dengan cara berikut:
> *conda install -c conda-forge jupyterlab* \
> *pip install cameo* \
> *pip install escher* \
> *conda install -c conda-forge nodejs* \
> *jupyter labextension install @jupyter-widgets/jupyterlab-manager* \
> *jupyter labextension install escher*

<a name="section3"></a>

## Database Genome Scale Metabolic Model
* Buka database: http://bigg.ucsd.edu/
* Cari model: **e_coli_core**
* Ada berapa banyak metabolit, reaksi, dan gen pada model tersebut?
* Genom apa yang digunakan sebagai dasar dari model tersebut?
* Reaksi & stoikiometri?
* Escher Map?
* Bandingkan [**e_coli_core**](http://bigg.ucsd.edu/models/e_coli_core) dengan [**iML1515**](http://bigg.ucsd.edu/models/iML1515). Apa perbedaan dari kedua model tersebut? Jumlah Metabolit, Reaksi, Gen? Referensi genom? Escher Map? 
<a name="section4"></a>

## Eksplorasi dan Simulasi Model

In [1]:
# load library
from cameo import models
from cameo import load_model
from cameo import fba
from escher import Builder

In [2]:
# load model e_coli_core
# model ini sudah tersedia di library
model = load_model('e_coli_core')

# ada berapa banyak metabolit & reaksi? Objektif? 
model

# bagaimana mengakses model yang ada di BIGG?
#models.index_models_bigg()
#models.bigg.iML1515

# anda juga bisa mengunduh model (format SBML) kemudian load model dengan menggunakan cameo

Name,e_coli_core
Memory address,0x07fba2371f370
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [3]:
# bagaimana mengakses komponen dalam model?
#model.genes

# ada berapa banyak gen? 
len(model.genes)

137

In [4]:
# load escher map - map kosong tanpa data
builder = Builder(
    map_name='e_coli_core.Core metabolism'
)
builder

Builder()

<img src="maps/01.svg">

In [5]:
# mari kita mencari tahu informasi terkait g3p_c, c disini menunjukkan lokasi metabolit 
model.metabolites.get_by_id('g3p_c')

# ada berapa reaksi yang menggunakan g3p_c?

Metabolite identifier,g3p_c
Name,Glyceraldehyde 3-phosphate
Memory address,0x07fba1cd08790
Formula,C3H5O6P
Compartment,c
In 7 reaction(s),"TKT1, GAPD, FBA, TPI, TALA, TKT2, BIOMASS_Ecoli_core_w_GAM"


In [6]:
# detail reaksi FBA - Fructose-bisphosphate aldolase
model.reactions.get_by_id('FBA')

Reaction identifier,FBA
Name,Fructose-bisphosphate aldolase
Memory address,0x07fba1cc2b640
Stoichiometry,"fdp_c <=> dhap_c + g3p_c D-Fructose 1,6-bisphosphate <=> Dihydroxyacetone phosphate + Glyceraldehyde 3-phosphate"
GPR,b2097 or b1773 or b2925
Lower bound,-1000.0
Upper bound,1000.0


In [7]:
# simulasi model
s = fba(model)
flux = s.fluxes.to_frame()
s.objective_value
#flux.loc['FBA', 'fluxes']

0.8739215069684303

In [8]:
# map fluxes
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=s.fluxes,
)
builder

Builder(reaction_data={'PFK': 7.477381962160287, 'PFL': 0.0, 'PGI': 4.860861146496827, 'PGK': -16.023526143167…

<img src="maps/02.svg">

<a name="section5"></a>

## Manipulasi Model
<a name="section5-1"></a>

### Membuat knock-out

In [9]:
# membuat copy dari model
mutant = model.copy()

# knockout reaksi FBA
mutant.reactions.FBA.knock_out()

# flux balance analysis pada mutan
s_mutant = fba(mutant)

# check perubahan flux
print('FBA bounds WT', model.reactions.FBA.bounds)
print('FBA bounds mutant', mutant.reactions.FBA.bounds)
print('growth rate WT:', s.objective_value)
print('growth rate mutant:',s_mutant.objective_value)

# map flux
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=s_mutant.fluxes,
)
builder

FBA bounds WT (-1000.0, 1000.0)
FBA bounds mutant (0, 0)
growth rate WT: 0.8739215069684303
growth rate mutant: 0.7040369478590354


Builder(reaction_data={'PFK': -7.647270926404825e-15, 'PFL': 0.0, 'PGI': -18.04341091820498, 'PGK': -8.8287641…

<img src="maps/03.svg">

<a name="section5-2"></a>

### Mengubah medium

In [10]:
# melihat medium
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

In [11]:
# mengubah medium menjadi anaerobik dengan menghilangkan O2
medium = model.medium
medium['EX_o2_e'] = 0.0000
model.medium = medium
s_anaerobic = fba(model)

# map flux
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=s_anaerobic.fluxes,
)
builder

Builder(reaction_data={'PFK': 9.789458863898286, 'PFL': 17.804674217935286, 'PGI': 9.95660909530426, 'PGK': -1…

<img src="maps/04.svg">

<a name="section5-3"></a>

### Mengubah objective

In [12]:
# return medium to normal
medium['EX_o2_e'] = 1000.0
model.medium = medium

print(model.objective.direction, model.objective.expression)

max 1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5


In [13]:
# kita coba ubah objektif untuk memaksimalkan ATP Maintenance
model.reactions.ATPM

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x07fba1ccc6a00
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,
Lower bound,8.39
Upper bound,1000.0


In [14]:
model.objective = model.reactions.ATPM
s_atpm = fba(model)

# map flux
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    reaction_data=s_atpm.fluxes,
)
builder

Builder(reaction_data={'PFK': 10.0, 'PFL': 0.0, 'PGI': 10.0, 'PGK': -20.0, 'PGL': 0.0, 'ACALD': 0.0, 'AKGt2r':…

<img src="maps/05.svg">